# Helpers borrowed from [Peter Norvig](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb)

In [1]:
import re
import math
from collections import OrderedDict, Counter, defaultdict, namedtuple, deque

from itertools   import permutations, combinations, chain, takewhile, \
cycle, product, islice, zip_longest, count as count_from

from heapq       import heappop, heappush

def Input(day):
    "Open this day's input file."
    filename = 'input_day{}.txt'.format(day)
    return open(filename).read().strip()

cat = ''.join
origin = (0,0)

def array(lines):
    "Convert an iterable of lines into a 2-D array. If lines is a str, do splitlines."
    if isinstance(lines, str): lines = lines.splitlines()
    return [mapt(atom, line.replace(",", " ").split()) for line in lines]

def atom(token):
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return token

def grep(pattern, lines):
    "Print lines that match pattern."
    for line in lines:
        if re.search(pattern, line):
            print(line)
            
def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

def cityblock_distance(p, q=origin): 
    "Manhatten distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def distance(p, q=origin): 
    "Hypotenuse distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

def X(point): return point[0]
def Y(point): return point[1]

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def always(value): return (lambda *args: value)

def Astar(start, moves_func, h_func, cost_func=always(1)):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    Path      = lambda s: ([] if (s is None) else Path(previous[s]) + [s])
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(s)
        for s2 in moves_func(s):
            g = path_cost[s] + cost_func(s, s2)
            if s2 not in path_cost or g < path_cost[s2]:
                heappush(frontier, (g + h_func(s2), s2))
                path_cost[s2] = g
                previous[s2] = s


# [Day 1](https://adventofcode.com/2017/day/1)

In [2]:
def captcha(input):
    ints = mapt(int, input)
    left = (2*ints)[1:][:len(ints)]
    pairs = zip(ints, left)
    same = filter(lambda p: p[0] == p[1], pairs)
    return sum(map(lambda p: p[0], same))

assert captcha('1122') == 3
assert captcha('1111') == 4
assert captcha('1234') == 0
assert captcha('91212129') == 9

captcha(Input(1))

1144

In [3]:
def captcha_half(input):
    ints = mapt(int, input)
    left = (2*ints)[int(len(ints)/2):][:len(ints)]
    pairs = zip(ints, left)
    same = filter(lambda p: p[0] == p[1], pairs)
    return sum(map(lambda p: p[0], same))

assert captcha_half('1212') == 6
assert captcha_half('1221') == 0
assert captcha_half('123425') == 4
assert captcha_half('123123') == 12
assert captcha_half('12131415') == 4

captcha_half(Input(1))

1194

# [Day 2](https://adventofcode.com/2017/day/2)

In [4]:
def spreadsheet_checksum(input):
    return sum([max(row) - min(row)
                for row in array(input)
               ])

spreadsheet_checksum(Input(2))

45351

In [5]:
def spreadsheet_checksum2(input):
    return sum([p[0] // p[1]
         for row in array(input)
         for p in permutations(row, 2) 
         if p[0] > p[1] 
         and p[0] % p[1] == 0])
        
spreadsheet_checksum2(Input(2))

275

# [Day 3](https://adventofcode.com/2017/day/3)

In [6]:
position = 277678

a = math.ceil((math.sqrt(position)))
if a % 2 == 0:
    a += 1
s = a * a - 4 * (a - 1) + 1
h = (a - 1) // 2
d = abs(((position - s + 1) % (a - 1)) - h)
h + d

475

In [7]:
def populate(m, p, f):
    cx, cy = p
    px = cx - 1
    py = cy
    
    while (cx - 1, cy) in m:
        m[(cx, cy)] = f(m, (px, py), (cx, cy))
        px = cx
        py = cy
        cy += 1
        
    while (cx, cy - 1) in m:
        m[(cx, cy)] = f(m, (px, py), (cx, cy))
        px = cx
        py = cy
        cx -= 1
        
    while (cx + 1, cy) in m:
        m[(cx, cy)] = f(m, (px, py), (cx, cy))
        px = cx
        py = cy
        cy -= 1
        
    while (cx, cy + 1) in m:
        m[(cx, cy)] = f(m, (px, py), (cx, cy))
        px = cx
        py = cy
        cx += 1
        
    return (cx, cy)

def mem_find(cell):
    n = (1, 0)
    mem = OrderedDict([((0, 0), 1)])
    
    while not cell in mem.values():
        n = populate(mem, n, lambda m, p, c: m[p] + 1)
        
    for p, c in mem.items():
        if c == cell:
            return p

p = mem_find(position)
cityblock_distance(p)


475

In [8]:
def neigh(m, p, c):
    return sum([m[n] for n in neighbors8(c) if n in m])
        
def mem_neigh_find(cell):
    n = (1, 0)
    mem = OrderedDict([((0, 0), 1)])
    
    while True:
        n = populate(mem, n, neigh)
        for p, c in mem.items():
            if c > cell:
                return c
        
mem_neigh_find(position)

279138

Different imlpementation based on complex numbers and a generator


In [9]:
U, D, R, L = 1j, -1j, 1, -1
turns = {R:U, U:L, L:D, D:R}

Cell = namedtuple('Cell', 'pos, dir, value')

def cdistance(point): 
    return abs(point.real) + abs(point.imag)

def cells(score):
    pos = 0
    direction = D
    v = 1
    occupied = {pos: v}
    
    while True:
        yield Cell(pos, direction, v)
        if not pos + turns[direction] in occupied:
            direction = turns[direction]
        
        pos += direction
        v = score(pos, v, occupied)
        occupied[pos] = v
            
def mem_find2():            
    for c in cells(lambda p, v, o: v + 1):
        if(c.value >= position):
            return int(cdistance(c.pos))

mem_find2()

475

In [10]:
def neigh8_c(p):
    return (p+R, p+L, p+U, p+D, p+R+U, p+R+D, p+L+U, p+L+D)

def mem_neigh_find2():            
    for c in cells(lambda p, v, o: sum([o[n] for n in neigh8_c(p) if n in o])):
        if(c.value >= position):
            return c.value

mem_neigh_find2()

279138

# [Day 4](https://adventofcode.com/2017/day/4)

In [11]:
def no_duplicates(passphrases):
    valid = []
    for passphrase in passphrases:
        if len(passphrase) == len(set(passphrase)):
            valid.append(passphrase)
            
    return valid
    
valid = no_duplicates(array(Input(4)))
len(valid)

466

In [12]:
def no_anagrams(passphrases):
    valid = []
    for passphrase in passphrases:
        ordred = mapt(cat, mapt(sorted, passphrase))
        if len(ordred) == len(set(ordred)):
            valid.append(passphrase)
        
    return valid
    
ana = no_anagrams(valid) 
len(ana)

251

# [Day 5](https://adventofcode.com/2017/day/5)

In [13]:
jumps = list(map(int, Input(5).split()))
pos = 0
steps = 0

while(pos < len(jumps)):
    p = pos
    pos += jumps[pos]
    jumps[p] += 1
    steps += 1
    
steps

358131

In [14]:
jumps = list(map(int, Input(5).split()))
pos = 0
steps = 0

while(pos < len(jumps)):
    p = pos
    pos += jumps[pos]
    if jumps[p] > 2:
        jumps[p] -= 1
    else:
        jumps[p] += 1  
    steps += 1
    
steps

25558839

# [Day 6](https://adventofcode.com/2017/day/6)

In [15]:
def get_cycles(banks):
    seen = set()

    while not tuple(banks) in seen:
        seen.add(tuple(banks))
        blocks = max(banks)
        i = banks.index(blocks)
        banks[i] = 0

        while blocks > 0:
            i += 1
            i %= len(banks)
            banks[i] += 1
            blocks -= 1

    return len(seen), banks

assert get_cycles([0, 2, 7, 0])[0] == 5

banks = list(array(Input(6))[0])
assert len(banks) == 16
cycles, end_banks = get_cycles(banks)
cycles

11137

In [16]:
loop, _ = get_cycles(end_banks)
loop

1037

# [Day 7](https://adventofcode.com/2017/day/7)

In [17]:
reports = array(Input(7))

def create_tree(reports):
    seen = set()
    children = set()
    tree = {}

    for report in reports:
        name = report[0]
        ch = report[3:]
        weight = int(report[1][1:-1])
        seen.add(name)
        if len(report) > 3:
            children |= set(ch)
            tree[name] = (weight, ch)
        else:
            tree[name] = (weight, [])
        
    return seen.difference(children).pop(), tree

root, tower = create_tree(reports)
root

'gynfwly'

In [18]:
def subtower_weight(start):
    stack = [start]
    weight = 0
    
    while stack:
        name = stack.pop()
        weight += tower[name][0]
        stack.extend(tower[name][1])
    
    return weight

stack = [root]
while stack:
    name = stack.pop()
    weights = {c: subtower_weight(c) for c in tower[name][1]}
    lc = Counter(weights.values()).most_common()[-1]

    if lc[1] == 1:
        diff = lc[0] - Counter(weights.values()).most_common()[0][0]
        stack.append([x for x, y in weights.items() if y == lc[0]][0])
        
tower[name][0] - diff

1526

# [Day 8](https://adventofcode.com/2017/day/8)

In [19]:
instructions = array(Input(8).split('\n'))

def interpret(instructions):
    registers = {}
    max_r = 0
    
    for i in instructions:
        out, op, v, cr = i[0], i[1], i[2], i[4]
        
        if not cr in registers:
            registers[cr] = 0
        if not out in registers:
            registers[out] = 0
            
        exp = str(registers[cr]) + cat(map(str, i[5:]))
            
        if eval(exp):
            if op == 'inc':
                registers[out] += int(v)
            else:
                registers[out] -= int(v)
                
        if registers[out] > max_r:
            max_r = registers[out]

    return max_r, registers
        
max_r, registers = interpret(instructions)
max(registers.values())

6012

In [20]:
max_r

6369

# [Day 9](https://adventofcode.com/2017/day/9)

In [21]:
tests = [
    ("{{{},{},{{}}}}", 16),
    ("{{<!!>},{<!!>},{<!!>},{<!!>}}", 9),
    ("{{<ab>},{<ab>},{<ab>},{<ab>}}", 9),
    ("{{<a!>},{<a!>},{<a!>},{<ab>}}", 3)
]

def groups(stream):
    stack, groups = [], []
    garbage, ignore = False, False
    score, garbage_size = 0, 0
    
    for i in range(len(stream)):
        if not garbage:
            if stream[i] == '{':
                stack.append(i)
            elif stream[i] == '}':
                score += len(stack)
                groups.append(stream[stack.pop():i+1])
            elif stream[i] == '<':
                garbage = True
        else:
            if not ignore:
                if stream[i] == '!':
                    ignore = True
                elif stream[i] == '>':
                    garbage = False
                else:
                    garbage_size += 1 
            else:
                ignore = False
            
    return score, garbage_size
  
for stream, score in tests:
    s, _ = groups(stream)
    assert s == score
    
groups(Input(9))

(8337, 4330)

# [Day 10](https://adventofcode.com/2017/day/10)

In [22]:
def split(li, b, l):
    b %= len(li)
    e = (b + l) % len(li)
    if e < b:
        return li[b:] + li[:e], li[e:b]
    else:
        return li[b:e], li[e:] + li[:b]

def hash(input, sl, repeat=1):
    skip, pos = 0, 0
    hash = [i for i in range(sl)]
    
    for _ in range(repeat):
        for i in input:
            a, b = split(hash, pos, i)
            hash = list(reversed(a)) + b
            hash = hash[sl - pos:] + hash[:sl - pos]

            pos = (pos + i + skip) % sl
            skip += 1

    return hash

input = "63,144,180,149,1,255,167,84,125,65,188,0,2,254,229,24"
test = hash([3, 4, 1, 5], 5)
assert test[0] * test[1] == 12
out1 = hash([int(v) for v in input.split(',')], 256)
out1[0] * out1[1]

4480

In [23]:
from functools import reduce
from binascii import hexlify

def sparse(input):
    bytes = [ord(c) for c in input] + [17, 31, 73, 47, 23]
    sparse = hash(bytes, 256, 64)
    dense = []
    for s in [sparse[i:i+16] for i in range(0, 256, 16)]:
        dense.append(reduce(lambda a,x: a^x, s))
    return str(hexlify(bytearray(dense)))[2:-1]
    
assert sparse("") == 'a2582a3a0e66e6e86e3812dcb672a272'
assert sparse("AoC 2017") == '33efeb34ea91902bb2f59c9920caa6cd'
assert sparse("1,2,3") == '3efbe78a8d82f29979031a4aa0b16a9d'
assert sparse("1,2,4") == '63960835bcdc130f0b66d7ff4f6a5a8e'
sparse(input)

'c500ffe015c83b60fad2e4b7d59dabc4'

# [Day 11](https://adventofcode.com/2017/day/11)

In [24]:
path = Input(11).split(',')

dirs = {'n':  (1, -1, 0),
        'ne': (0, -1, 1),
        'se': (-1, 0, 1),
        's':  (-1, 1, 0),
        'sw': (0, 1, -1),
        'nw': (1, 0, -1)
        }

def hex_dist(point):
    x, y, z = point
    return (abs(x) + abs(y) + abs(z)) // 2
            
def hex_find(path):
    maxp = (0, 0, 0)
    goal = (0, 0, 0)
    for d in path:
        dx, dy, dz = dirs[d]
        goal = (goal[0]+dx, goal[1]+dy, goal[2]+dz)
        if hex_dist(goal) > hex_dist(maxp):
            maxp = goal

    return hex_dist(goal), hex_dist(maxp)

hex_find(path)


(643, 1471)

# [Day 12](https://adventofcode.com/2017/day/12)

In [25]:
test_input = """
0 <-> 2
1 <-> 1
2 <-> 0, 3, 4
3 <-> 2, 4
4 <-> 2, 3, 6
5 <-> 6
6 <-> 4, 5
""".strip()

def get_entries(input):
    entries = {entry[0]: set(entry[2:]) for entry in array(input)}         
    return entries

def group(pid, entries):

    stack = [pid]
    g = frozenset()

    while stack:
        pid = stack.pop()
        stack.extend(list(entries[pid].difference(g)))
        g |= entries[pid]

    return g

test_entries = get_entries(test_input)
input_entries = get_entries(Input(12))
    
assert len(group(0, test_entries)) == 6
len(group(0, input_entries))

380

In [26]:
def total(entries):
    groups = set(group(pid, entries) for pid in entries.keys())
    return len(groups)
        
assert total(test_entries) == 2
%time total(input_entries)

CPU times: user 856 ms, sys: 0 ns, total: 856 ms
Wall time: 857 ms


181

# [Day 13](https://adventofcode.com/2017/day/13)

In [27]:
test_input = '''
0: 3
1: 2
4: 4
6: 4
'''.strip()

def get_severity(firewall, delay=0):
    severity = 0
    caught = 0
    s = max(firewall.keys())+1
    for p in range(s):
        if p in firewall:
            if (delay + p) % (2 * (firewall[p] - 1)) == 0:
                severity += p*firewall[p]
                caught += 1
            
    return severity, caught

test_firewall = {d:r for d,r in array(test_input.replace(':', ''))}
firewall = {d:r for d,r in array(Input(13).replace(':', ''))}
assert get_severity(test_firewall)[0] == 24
get_severity(firewall)[0]

2508

In [28]:
assert get_severity(test_firewall, 10)[0] == 0

delay = 0
while True:
    _, c = get_severity(firewall, delay)
    if c == 0:
        break
    delay += 1
delay

3913186